In [2]:
#import libraries
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', -1)
import os

In [3]:
os.getcwd()

'/Users/akankshamishra/Downloads'

In [4]:
os.chdir("/Users/akankshamishra/Desktop/hackathon 2/data")

In [5]:
#Load csv
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train["test/train"] = "train"
train=train.dropna()
#print(train.head(10))
test["test/train"] = "test"
train_corpus = train[['tweet','test/train']]
test_corpus = test[['tweet','test/train']]
corpus = pd.concat([train_corpus,test_corpus],ignore_index=True)
print(corpus.tail(10))

                                                                                                                            tweet  \
9082  #Apple to Hawk iPad 2 at #SXSW Festival Popup Store!  {link}  by @mention                                                     
9083  Hey @mention ask Dennis why no iPad app?  #SXSW #checkins                                                                     
9084  RT @mention Find where @mention is playing at #SXSW on the #eventseekr app for Android. We got u! {link}                      
9085  It's not a rumor: Apple is opening up a temporary store in downtown Austin for #SXSW and the iPad 2 launch {link}             
9086  Apple store coming 2 Austin for SXSW.   RT: &quot;@mention Before It Even Begins, Apple Wins #SXSW {link} on @mention         
9087  @mention @mention @mention Hmmm....how fast can #apple build a new store in time for #sxsw  {link}                            
9088  Samsung Galaxy S II Appears At FCC And Team Android #SXSW Party

In [6]:
type(corpus)
text = corpus["tweet"][2]
print(text)
print(type(text))
text = train["tweet"][2]
print(text)

one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At #SXSW, Apple schools the mkt experts&quot;  {link}
<class 'str'>
one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At #SXSW, Apple schools the mkt experts&quot;  {link}


In [7]:
#train_corpus = train["tweet"].tolist()
#train_label = train["sentiment"].tolist()
#test_corpus = test["tweet"].tolist()
#test_label = test["sentiment"].tolist()
#data = train_corpus + test_corpus

In [8]:
print(train.head(10))
print("Test dataset")
print(test.head(10))

   tweet_id  \
0  1701       
1  1851       
2  2689       
3  4525       
4  3604       
5  966        
6  1395       
7  8182       
8  8835       
9  883        

                                                                                                                                             tweet  \
0  #sxswnui #sxsw #apple defining language of touch with different dialects becoming smaller                                                         
1  Learning ab Google doodles! All doodles should be light, funny &amp; innovative, with exceptions for significant occasions. #GoogleDoodle #sxsw   
2  one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At #SXSW, Apple schools the mkt experts&quot;  {link}              
3  This iPhone #SXSW app would b pretty awesome if it didn't crash every 10mins during extended browsing. #Fuckit #Illmakeitwork                     
4  Line outside the Apple store in Austin waiting for the new iPad #SXSW  {link}    

In [9]:
#VALUE COUNTS
#0 -> Negative sentiment
#1 -> Neutral sentiment
#2 -> Positive sentiment
#3 -> Can't tell
train['sentiment'].value_counts()

1    4310
2    2382
0    456 
3    125 
Name: sentiment, dtype: int64

In [10]:
#One blank tweet deleting it
train.isnull().sum()

tweet_id      0
tweet         0
sentiment     0
test/train    0
dtype: int64

In [11]:
test.isnull().sum()

tweet_id      0
tweet         0
test/train    0
dtype: int64

In [12]:
train.dropna()

,tweet_id,tweet,sentiment,test/train
0,1701,#sxswnui #sxsw #apple defining language of touch with different dialects becoming smaller,1,train
1,1851,"Learning ab Google doodles! All doodles should be light, funny &amp; innovative, with exceptions for significant occasions. #GoogleDoodle #sxsw",1,train
2,2689,"one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At #SXSW, Apple schools the mkt experts&quot; {link}",2,train
3,4525,This iPhone #SXSW app would b pretty awesome if it didn't crash every 10mins during extended browsing. #Fuckit #Illmakeitwork,0,train
4,3604,Line outside the Apple store in Austin waiting for the new iPad #SXSW {link},1,train
5,966,#technews One lone dude awaits iPad 2 at AppleÛªs SXSW store {link} #Tech_News #Apple #iPad_2 #SXSW #tablets #tech,1,train
6,1395,"SXSW Tips, Prince, NPR Videos, Toy Shopping With Zuckerberg.\r\n{link} #sxsw #ipad",1,train
7,8182,NU user RT @mention New #UberSocial for #iPhone now in the App Store includes UberGuide to #SXSW sponsored by #Mashable,1,train
8,8835,Free #SXSW sampler on iTunes {link} #FreeMusic,2,train
9,883,I think I might go all weekend without seeing the same iPad case twice... #sxsw,2,train


In [13]:
train.shape

(7273, 4)

In [14]:
train = train.dropna()

In [15]:
train.shape

(7273, 4)

In [16]:
print(train.shape)
print(test.shape)
print(train.isnull().sum())
print(test.isnull().sum())
print(corpus.shape)
print(corpus["test/train"][7272])

(7273, 4)
(1819, 3)
tweet_id      0
tweet         0
sentiment     0
test/train    0
dtype: int64
tweet_id      0
tweet         0
test/train    0
dtype: int64
(9092, 2)
train


In [24]:
stuff_to_be_removed = list(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
cleaned_tweet=[]
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

def tweet_cleanining(tweet):
    clean_tweet=[]
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    #Replace {}
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    #clean_tweet = "Tweet cleaned"
    #Cleaning for each word
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word) and word not in stuff_to_be_removed:
        #    if use_stemmer:
        #        word = str(porter_stemmer.stem(word))
        #    word = word_tokenize(word)
            word = lemmatizer.lemmatize(word)
            clean_tweet.append(word)

    return ' '.join(clean_tweet)

#new_train = tweet_cleanining(train)

#test_df = train[:30]
for ind in corpus.index: 
    #print('Text before processing')
    #text = corpus['tweet'][ind]
    text = tweet_cleanining(corpus['tweet'][ind])
    cleaned_tweet.append(text)
    #train['tweet'][ind] = text
    #print('Text after cleaning',text)

corpus['cleaned_tweet'] = cleaned_tweet   
#print(corpus['tweet'],corpus['cleaned_tweet'])    
# For Test
#test_cleaned_tweet=[]
#for ind in test.index: 
    #print('Text before processing')
    #print(train['tweet'][ind])
   # text = tweet_cleanining(test['tweet'][ind])
    #test_cleaned_tweet.append(text)
    #train['tweet'][ind] = text
    #print('Text after cleaning',text)

#test['cleaned_tweet'] = cleaned_tweet   
#print(train['tweet'],train['cleaned_tweet']) 

#new_train.head(10)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [26]:
cv = CountVectorizer()
tfidf = TfidfVectorizer(stop_words="english")

In [27]:
train.columns

Index(['tweet_id', 'tweet', 'sentiment', 'test/train'], dtype='object')

In [28]:
#vector = tfidf.fit_transform(new_train["cleaned_tweet"])
vector = tfidf.fit_transform(corpus["cleaned_tweet"])

In [29]:
print(train.shape)
print(corpus['test/train'][7273],corpus['test/train'][7272])

(7273, 4)
test train


In [30]:
corpus_X = pd.DataFrame(vector.toarray(),columns=list(tfidf.get_feature_names()))

In [31]:
X = corpus_X[0:7273]
X_submission = corpus_X[7273:]

In [32]:
print(X.shape)
print(X_submission.shape)

(7273, 8268)
(1819, 8268)


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier

In [42]:
y = train['sentiment']

In [43]:
X_train, X_test, y_train, y_test = tts(X,y,random_state=42,test_size=0.20)

In [44]:
rf = OneVsRestClassifier(RandomForestClassifier(random_state=10))
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_rf = rf.score(X_test,y_test)
confusion_mat_rf = confusion_matrix(y_test,y_pred)
class_repo_rf = classification_report(y_test,y_pred)
print(accuracy_rf)
print(confusion_mat_rf)
print(class_repo_rf)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6721649484536083
[[ 15  58  13   2]
 [  8 743  94   4]
 [  2 272 219   2]
 [  2  18   2   1]]
              precision    recall  f1-score   support

           0       0.56      0.17      0.26        88
           1       0.68      0.88      0.77       849
           2       0.67      0.44      0.53       495
           3       0.11      0.04      0.06        23

    accuracy                           0.67      1455
   macro avg       0.50      0.38      0.41      1455
weighted avg       0.66      0.67      0.64      1455



In [45]:
sgd = OneVsRestClassifier(SGDClassifier(max_iter=5, tol=None))
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
accuracy_sgd = sgd.score(X_test,y_test)
confusion_mat_sgd = confusion_matrix(y_test,y_pred)
class_repo_sgd = classification_report(y_test,y_pred)
print(accuracy_sgd)
print(confusion_mat_sgd)
print(class_repo_sgd)

0.6790378006872853
[[ 20  59   8   1]
 [  5 797  44   3]
 [  3 320 171   1]
 [  1  20   2   0]]
              precision    recall  f1-score   support

           0       0.69      0.23      0.34        88
           1       0.67      0.94      0.78       849
           2       0.76      0.35      0.47       495
           3       0.00      0.00      0.00        23

    accuracy                           0.68      1455
   macro avg       0.53      0.38      0.40      1455
weighted avg       0.69      0.68      0.64      1455



In [46]:
lr = OneVsRestClassifier(LogisticRegression(random_state=10))
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
accuracy_lr = lr.score(X_test,y_test)
confusion_mat_lr = confusion_matrix(y_test,y_pred)
class_repo_lr = classification_report(y_test,y_pred)
print(accuracy_lr)
print(confusion_mat_lr)
print(class_repo_lr)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6707903780068728
[[  0  76  12   0]
 [  0 797  52   0]
 [  0 316 179   0]
 [  0  21   2   0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        88
           1       0.66      0.94      0.77       849
           2       0.73      0.36      0.48       495
           3       0.00      0.00      0.00        23

    accuracy                           0.67      1455
   macro avg       0.35      0.33      0.31      1455
weighted avg       0.63      0.67      0.62      1455



//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
gnb = OneVsRestClassifier(MultinomialNB())
gnb.fit(X_train,y_train)
y_pred = gnb.predict(X_test)
accuracy_gnb = gnb.score(X_test,y_test)
confusion_mat_gnb = confusion_matrix(y_test,y_pred)
class_repo_gnb = classification_report(y_test,y_pred)
print(accuracy_gnb)
print(confusion_mat_gnb)
print(class_repo_gnb)

0.6364261168384879
[[  1  77  10   0]
 [  0 810  39   0]
 [  0 380 115   0]
 [  0  21   2   0]]
              precision    recall  f1-score   support

           0       1.00      0.01      0.02        88
           1       0.63      0.95      0.76       849
           2       0.69      0.23      0.35       495
           3       0.00      0.00      0.00        23

    accuracy                           0.64      1455
   macro avg       0.58      0.30      0.28      1455
weighted avg       0.66      0.64      0.56      1455



//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
p = OneVsRestClassifier(Perceptron(max_iter=5))
p.fit(X_train,y_train)
y_pred = p.predict(X_test)
accuracy_p = p.score(X_test,y_test)
confusion_mat_p = confusion_matrix(y_test,y_pred)
class_repo_p = classification_report(y_test,y_pred)
print(accuracy_p)
print(confusion_mat_p)
print(class_repo_p)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


0.6460481099656358
[[ 14  48  19   7]
 [  9 667 149  24]
 [  1 222 258  14]
 [  1  17   4   1]]
              precision    recall  f1-score   support

           0       0.56      0.16      0.25        88
           1       0.70      0.79      0.74       849
           2       0.60      0.52      0.56       495
           3       0.02      0.04      0.03        23

    accuracy                           0.65      1455
   macro avg       0.47      0.38      0.39      1455
weighted avg       0.65      0.65      0.64      1455



//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [49]:
svm = OneVsRestClassifier(LinearSVC(random_state=10))
svm.fit(X_train,y_train)
y_pred = svm.predict(X_test)
accuracy_svm = svm.score(X_test,y_test)
confusion_mat_svm = confusion_matrix(y_test,y_pred)
class_repo_svm = classification_report(y_test,y_pred)
print(accuracy_svm)
print(confusion_mat_svm)
print(class_repo_svm)

0.6893470790378007
[[ 21  48  18   1]
 [  9 726 107   7]
 [  6 232 255   2]
 [  2  16   4   1]]
              precision    recall  f1-score   support

           0       0.55      0.24      0.33        88
           1       0.71      0.86      0.78       849
           2       0.66      0.52      0.58       495
           3       0.09      0.04      0.06        23

    accuracy                           0.69      1455
   macro avg       0.50      0.41      0.44      1455
weighted avg       0.68      0.69      0.67      1455



In [50]:
dt = OneVsRestClassifier(DecisionTreeClassifier(random_state=10))
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
accuracy_dt = dt.score(X_test,y_test)
confusion_mat_dt = confusion_matrix(y_test,y_pred)
class_repo_dt = classification_report(y_test,y_pred)
print(accuracy_dt)
print(confusion_mat_dt)
print(class_repo_dt)

0.5532646048109966
[[ 12  35  19  22]
 [  9 571 134 135]
 [  5 191 212  87]
 [  0  11   2  10]]
              precision    recall  f1-score   support

           0       0.46      0.14      0.21        88
           1       0.71      0.67      0.69       849
           2       0.58      0.43      0.49       495
           3       0.04      0.43      0.07        23

    accuracy                           0.55      1455
   macro avg       0.45      0.42      0.37      1455
weighted avg       0.64      0.55      0.58      1455



In [51]:
adb = AdaBoostClassifier(random_state=10)
adb.fit(X_train,y_train)
y_pred = adb.predict(X_test)
accuracy_adb = adb.score(X_test,y_test)
confusion_mat_adb = confusion_matrix(y_test,y_pred)
class_repo_adb = classification_report(y_test,y_pred)
print(accuracy_adb)
print(confusion_mat_adb)
print(class_repo_adb)

0.6048109965635738
[[  6  67  14   1]
 [  3 797  44   5]
 [  1 417  76   1]
 [  0  22   0   1]]
              precision    recall  f1-score   support

           0       0.60      0.07      0.12        88
           1       0.61      0.94      0.74       849
           2       0.57      0.15      0.24       495
           3       0.12      0.04      0.06        23

    accuracy                           0.60      1455
   macro avg       0.48      0.30      0.29      1455
weighted avg       0.59      0.60      0.52      1455



In [52]:
acc_df = pd.DataFrame({
    'Model': ['Support Vector Machines','Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree',"AdaBoost"],
    'Score': [accuracy_svm, accuracy_lr, 
              accuracy_rf, accuracy_gnb, accuracy_p, 
              accuracy_sgd, accuracy_dt,accuracy_adb]})
acc_df = acc_df.sort_values(by='Score', ascending=False)
acc_df = acc_df.set_index('Score')
acc_df

,Model
Score,
0.689347,Support Vector Machines
0.679038,Stochastic Gradient Decent
0.672165,Random Forest
0.670790,Logistic Regression
0.646048,Perceptron
0.636426,Naive Bayes
0.604811,AdaBoost
0.553265,Decision Tree


In [55]:
X_submission.head()

,a11y2go,aapl,aaron,ab,abacus,abandoned,abba,aber,ability,able,...,zlf,zms,zomb,zombie,zomg,zone,zoom,zuckerberg,zynga,zzs
7273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
pred_rf = rf.predict(X_submission)
result = pd.DataFrame(dict(zip(X_submission.index,pred_rf)).items(), columns = ["tweet_id", "sentiment"])
result.to_csv("/Users/akankshamishra/Desktop/test_submission_new/result_rf.csv", index= False)

In [59]:
pred_dt = dt.predict(X_submission)
result = pd.DataFrame(dict(zip(X_submission.index,pred_dt)).items(), columns = ["tweet_id", "sentiment"])
result.to_csv("/Users/akankshamishra/Desktop/test_submission_new/result_dt.csv", index= False)

In [62]:
pred_adb = adb.predict(X_submission)
result = pd.DataFrame(dict(zip(X_submission.index,pred_adb)).items(), columns = ["tweet_id", "sentiment"])
result.to_csv("/Users/akankshamishra/Desktop/test_submission_new/result_adb.csv", index= False)

In [63]:
pred_lr = lr.predict(X_submission)
result = pd.DataFrame(dict(zip(X_submission.index,pred_lr)).items(), columns = ["tweet_id", "sentiment"])
result.to_csv("/Users/akankshamishra/Desktop/test_submission_new/result_lr.csv", index= False)

In [64]:
pred_p = p.predict(X_submission)
result = pd.DataFrame(dict(zip(X_submission.index,pred_p)).items(), columns = ["tweet_id", "sentiment"])
result.to_csv("/Users/akankshamishra/Desktop/test_submission_new/result_p.csv", index= False)

In [65]:
pred_svm = svm.predict(X_submission)
result = pd.DataFrame(dict(zip(X_submission.index,pred_svm)).items(), columns = ["tweet_id", "sentiment"])
result.to_csv("/Users/akankshamishra/Desktop/test_submission_new/result_svm.csv", index= False)

In [66]:
pred_sgd = sgd.predict(X_submission)
result = pd.DataFrame(dict(zip(X_submission.index,pred_sgd)).items(), columns = ["tweet_id", "sentiment"])
result.to_csv("/Users/akankshamishra/Desktop/test_submission_new/result_sgd.csv", index= False)

In [67]:
pred_gnb = gnb.predict(X_submission)
result = pd.DataFrame(dict(zip(X_submission.index,pred_gnb)).items(), columns = ["tweet_id", "sentiment"])
result.to_csv("/Users/akankshamishra/Desktop/test_submission_new/result_gnb.csv", index= False)